In [ ]:
from sklearn.tree._tree import TREE_LEAF
import numpy as np

class DtRules2Code:
    def generate_sql_code(self, class_names=None):
        idx = np.argwhere(self.left == TREE_LEAF)[:, 0]

        def get_node_path(left, right, child, lineage=None):
            if lineage is None:
                lineage = [child]

            if child in left:
                parent = np.where(left == child)[0][0]
                split = 'l'
            else:
                parent = np.where(right == child)[0][0]
                split = 'r'
            lineage.append((parent, split, self.threshold[parent], self.features[parent]))

            if parent == 0:
                lineage.reverse()
                return lineage
            else:
                return get_node_path(left, right, parent, lineage)

        print('CASE ')
        for j, child in enumerate(idx):
            clause = '  WHEN '
            for node in get_node_path(self.left, self.right, child):
                if not isinstance(node, tuple):
                    continue
                i = node
                if i[1] == 'l' : sign = self.le
                else: sign = self.gt
                clause = clause + i[3] + sign + str(i[2]) + ' AND '

            clause = clause[:-4] + ' THEN ' + str(np.argmax(self.value[child]))
            print(clause)
        print('ELSE -1 END')


In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline

In [2]:
data = load_iris()
X = data.data
y = data.target
pipeline = PMMLPipeline([("classifier", DecisionTreeClassifier(max_depth=8))])
pipeline.fit(X, y)
sklearn2pmml(pipeline, "example.pmml", with_repr=True)

C:\Users\Administrator\anaconda3\lib\subprocess.py:848: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
C:\Users\Administrator\anaconda3\lib\subprocess.py:853: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [4]:
import treelite
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
import xgboost
dtrain = xgboost.DMatrix(X, label=y)
params = {'max_depth': 3, 'eta': 1, 'silent': 1, 'objective': 'reg:linear', 'eval_metric':'rmse'}
bst = xgboost.train(params, dtrain, 20, [(dtrain, 'train')])

[16:46:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:46:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:3.89049
[1]	train-rmse:3.38204
[2]	train-rmse:3.10513
[3]	train-rmse:2.84322
[4]	train-rmse:2.60580
[5]	train-rmse:2.45425
[6]	train-rmse:2.29526
[7]	train-rmse:2.17920
[8]	train-rmse:2.09359
[9]	train-rmse:1.96872
[10]	train-rmse:1.93416
[11]	train-rmse:1.83528
[12]	train-rmse:1.78750
[13]	train-rmse:1.71018
[14]	train-rmse:1.64747
[15]	train-rmse:1.57359
[16]	train-rmse:1.49626
[17]	train

In [5]:
model = treelite.Model.from_xgboost(bst)
# install gcc first
model.export_lib(toolchain='gcc', libpath='./mymodel.so', verbose=True)

ValueError: Toolchain gcc not found. Ensure that it is installed and that it is a variant of GCC or Clang.

In [ ]:
import treelite_runtime
predictor = treelite_runtime.predictor('./mymodel.so', verbose=True)
# bath = treelite_runtime.Batch.fromnpy2d(X, rbegin=0, rend=5)
# predictor.predict(batch)